# Ludwig test


``` bash
# Get ludwig
#Library prereq for ludwig 2
sudo apt-get install libgmp3-dev
sudo apt-get install libsndfile1
# Now install ludwig (version 0.2 works with tensorflow 1.14 on python 
pip3 install ludwig==0.2
```

## Splitting Moby Dick and Romeo and Juliet

``` bash
# Word wrap at 80 characters
fold -w 80 -s moby_dick.txt

# Now split the text every 5 lines, and prefix with a label
cd book_data
split -d -l 5 ../romeo_juliet.txt RJ_
split -d -l 5 ../moby_dick.txt MD_
```

## Key Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import subprocess
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline

## Import our Labeled Text into a CSV (via DataFrame)

In [2]:
# First, let's import the full collection of labeled texts
#
# All of the text is in "book_data" and begins with a label "MD_" for "Moby Dick" 
# or "RJ_" for Romeo and Juliet
#

filenames = []
texts = []
labels = []
path = "book_data"
# First let's
for file in os.listdir(path):
    if not file.startswith('.'):
        filenames.append(file)
        labels.append(file.split('_')[0])
        with open(os.path.join(path,file), 'r') as f:
            texts.append(f.read())

# We read the text and insert it into a pandas dataframe. Sklearn uses dataframes
# because it is a table class with nice analysis methods
labeled_data = pd.DataFrame()
labeled_data['text'] = texts
labeled_data['label'] = labels

# Display the first 6 rows to show that we successfully read it
labeled_data.head(6)

,text,label
0,"“Cook,” said Stubb, rapidly lifting a rather r...",MD
1,"\nA short space elapsed, and up into this nois...",MD
2,"said humorous Stubb one day, “he can never be ...",MD
3,and portion off their nieces with a few porpoi...,MD
4,as ordinary fish possess what is called a swim...,MD
5,"JULIET\nNow, good sweet nurse,--O Lord, why lo...",RJ


### Save as a CSV

In [3]:
# Now save the data as a csv file
labeled_data.to_csv("training_set.csv", index=False)
! head training_set.csv

text,label
"“Cook,” said Stubb, rapidly lifting a rather reddish morsel to his mouth, 
“don’t you think this steak is rather overdone? You’ve been beating this 
steak too much, cook; it’s too tender. Don’t I always say that to be good, 
a whale-steak must be tough? There are those sharks now over the side, don’t 
you see they prefer it tough and rare? What a shindy they are kicking up! Cook, 
",MD
"
A short space elapsed, and up into this noiselessness came Ahab alone from his 
cabin. Taking a few turns on the quarter-deck, he paused to gaze over the side, 


## Use Ludwig to Train a Model

In [ ]:
!date
!ludwig experiment \
--data_csv training_set.csv \
--model_definition_file model_definition.yaml \
--output_directory ./results
!date


Mon Oct 12 15:00:56 EDT 2020
/home/butch/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/butch/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/butch/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/butch/.local/lib/python3.6/site-packages/tensorflow/python/






Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor






╒══════════╕
│ TRAINING │
╘══════════╛

2020-10-12 15:00:59.394500: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-10-12 15:00:59.419320: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3801000000 Hz
2020-10-12 15:00:59.419545: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x692b580 executing computations on platform Host. Devices:
2020-10-12 15:00:59.419567: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2020-10-12 15:00:59.628038: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use exp



Epoch   7
Evaluation test : 100%|███████████████████████████| 8/8 [00:01<00:00,  6.54it/s]
Took 26.8506s
╒═════════╤════════╤════════════╤═════════════╕
│ label   │   loss │   accuracy │   hits_at_k │
╞═════════╪════════╪════════════╪═════════════╡
│ train   │ 0.0001 │     1.0000 │      1.0000 │
├─────────┼────────┼────────────┼─────────────┤
│ vali    │ 0.0381 │     0.9887 │      1.0000 │
├─────────┼────────┼────────────┼─────────────┤
│ test    │ 0.0302 │     0.9893 │      1.0000 │
╘═════════╧════════╧════════════╧═════════════╛
Last improvement of loss on combined happened 3 epochs ago


Epoch   8
Training:  22%|███████▌                          | 6/27 [00:04<00:16,  1.29it/s]

In [ ]:
# Run visualize
!ludwig visualize -v learning_curves -trs results/experiment_run_0/training_statistics.json -tes results/experiment_run_0/test_statistics.json -od .


In [ ]:
!ls *.pdf

In [ ]:
from IPython.display import IFrame
IFrame("learning_curves_combined_accuracy.pdf", width=600, height=500)

In [ ]:
IFrame("learning_curves_combined_loss.pdf", width=600, height=500)